In [1]:
import pyspark
import pymongo
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession
from transformers import pipeline
from pyspark.sql.functions import mean, count, max, min

In [ ]:
spark.stop()

In [2]:
spark = SparkSession.builder.appName("Sentiment").config("spark.driver.extraClassPath","/usr/local/spark/jars/mysql-connector-java-8.0.33.jar").getOrCreate()

23/10/20 21:50:44 WARN Utils: Your hostname, muhammad-Vm resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/10/20 21:50:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 21:50:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sentiment_analyzer = pipeline("sentiment-analysis", model = "distilbert-base-uncased-finetuned-sst-2-english")

In [4]:
def get_sentiment_score(text):
    result = sentiment_analyzer(text)
    score = result[0]["score"]
    return score

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [ ]:
def get_sentiment_score(text):
    # tokenise the text and convert to tensor
    inputs = tonkenizer(text, return_tensors = "pt")
    # feed the iputs to the model and get the logits as output
    with torch.no_grad():
        logits = model(**inputs).logits
    # get the preficited class IS by taking the index of the maximum value in the logits vector
    predicted_class_id = logits.argmax().item()
    # get the sentiment score by using the model.config.id2label dictionary, which maps class IDs to sentiment scores
    score = model.config.id2label[predicted_class_id]
    return score

In [5]:
spark.udf.register("get_sentiment_score", get_sentiment_score)

<function __main__.get_sentiment_score(text)>

In [6]:
jdbc_url = "jdbc:mysql://localhost:3306/Tweets"
connection_properties = {
    "user":"root",
    "password":"password",
    "driver":"com.mysql.cj.jdbc.Driver"
}

In [7]:
df = spark.read.jdbc(
    url = jdbc_url, 
    table = "tweetable1", 
    properties = connection_properties
)

In [ ]:
df.createOrReplaceTempView("view")

In [ ]:
result = spark.sql("SELECT * FROM view")

In [8]:
df = df.withColumn("sentiment_score", udf(get_sentiment_score)(df["text"]))

In [9]:
df = df.groupBy("date").agg(
    mean("sentiment_score").alias("mean_sentiment_score"),
    count("sentiment_score").alias("count_sentiment_score"),
    max("sentiment_score").alias("max_sentiment_score"),
    min("sentiment_score").alias("min_sentiment_score")
)

df = df.withColumn("sentiment_range", df["max_sentiment_score"] - df["min_sentiment_score"])
df = df.select("date", "mean_sentiment_score", "count_sentiment_score", "sentiment_range")

In [ ]:
df.write.format("mongo").mode("append").option("database", "database_name").option("collection", "collection_name").save()

In [10]:
# Set the MongoDB connection details
mongo_uri = "mongodb://localhost:27017"
database_name = "my_database"
collection_name = "my_collection"

# Write the DataFrame to MongoDB
df.write.format("mongo").mode("append").option("uri", mongo_uri).option("database", database_name).option("collection", collection_name).save()

Py4JJavaError: An error occurred while calling o98.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: mongo. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:248)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(Delegatin